## Descripción del problema:

El objetivo de este libro es obtener un modelo que sea capaz de reconocer los dígitos escritos a mano

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

2024-06-28 10:13:48.481167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
df = pd.read_csv('train.csv')
df.shape

(42000, 785)

In [3]:
# Datos de entrenamiento y validación
y = df['label']
X = df.drop(['label'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y,
          train_size=0.9, test_size=0.1, random_state=0)
x_train, x_test, y_train, y_test = x_train.values, x_test.values, y_train.values, y_test.values
x_train, x_test = x_train/255, x_test/255

In [4]:
f"Conjunto de entrenamiento: {x_train.shape} y {y_train.shape}" \
f" y validación: {x_test.shape} y {y_test.shape}"

'Conjunto de entrenamiento: (37800, 784) y (37800,) y validación: (4200, 784) y (4200,)'

In [5]:
x_train = np.reshape(x_train, (37800, 28, 28))
x_test = np.reshape(x_test,(4200, 28, 28))
f"Conjunto de entrenamiento: {x_train.shape} y {y_train.shape}" \
f" y validación: {x_test.shape} y {y_test.shape}"

'Conjunto de entrenamiento: (37800, 28, 28) y (37800,) y validación: (4200, 28, 28) y (4200,)'

In [6]:
model = models.Sequential([
  layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64, (3,3), activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64, (3,3), activation='relu'),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(10, activation='softmax'),
])
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics= ['accuracy']
)

/Users/asaucedov/anaconda3/envs/python_3_12_3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.8622 - loss: 0.4374
Epoch 2/5
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9817 - loss: 0.0600
Epoch 3/5
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - accuracy: 0.9883 - loss: 0.0390
Epoch 4/5
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - accuracy: 0.9907 - loss: 0.0295
Epoch 5/5
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.9926 - loss: 0.0226


In [8]:
# Evaluación del modelo
model.evaluate(x_test, y_test, verbose=2)

132/132 - 1s - 5ms/step - accuracy: 0.9893 - loss: 0.0416


[0.0416322685778141, 0.9892857074737549]

In [15]:
# Predicciones (ImageId, label)
df_test = pd.read_csv('test.csv')
predict_test = model.predict(df_test.values.reshape((28000, 28, 28)))
type(predict_test)

875/875 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


numpy.ndarray

In [17]:
predict_test_labels = np.argmax(predict_test, axis=1)
predict_test_labels.shape

(28000,)

In [23]:
results = {
  "ImageId": df_test.index.values + 1,
  "Label": predict_test_labels,
}
results

{'ImageId': array([    1,     2,     3, ..., 27998, 27999, 28000]),
 'Label': array([2, 0, 9, ..., 3, 9, 2])}

In [24]:
results_df = pd.DataFrame(results)
results_df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [25]:
results_df.to_csv('results.csv', index=False)